In [1]:
# ==============================================================================
# DISCLAIMER IMPORTANTE PARA AMBIENTES GOOGLE COLAB
# ==============================================================================
# Como o Google Colab fornece um ambiente de execução temporário, ele não vem
# com Java ou Spark pré-instalados. Portanto, este bloco de código DEVE SER
# EXECUTADO TODA VEZ que você iniciar ou reiniciar uma sessão (runtime) no Colab
# para instalar e configurar todas as dependências necessárias para o PySpark.
# ==============================================================================


# --- Bloco de Instalação e Configuração do Ambiente Spark ---

# 1. Instalação do Java
# O Apache Spark é executado sobre a Java Virtual Machine (JVM), então o Java é um pré-requisito obrigatório.
# !apt-get update -qq: Atualiza a lista de pacotes do sistema operacional (baseado em Debian/Ubuntu). O '-qq' torna a saída mais silenciosa.
!apt-get update -qq
# !apt-get install: Instala o OpenJDK 11 (uma versão de código aberto do Java) sem interação do usuário (-y).
!apt-get install -y openjdk-11-jdk-headless

# 2. Download do Spark
# Baixa os binários pré-compilados do Apache Spark a partir do site oficial de arquivamento da Apache.
# !wget -q: Baixa o arquivo da URL especificada. O '-q' (quiet) minimiza as mensagens de log durante o download.
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

# 3. Extração do Spark
# Descompacta o arquivo .tgz que foi baixado no passo anterior.
# !tar -xvzf:
#   x: eXtract (extrair)
#   v: verbose (mostra os arquivos sendo extraídos)
#   z: gZip (indica que o arquivo está compactado com gzip)
#   f: file (especifica o nome do arquivo a ser descompactado)
!tar -xvf spark-3.5.1-bin-hadoop3.tgz

# 4. Configuração das Variáveis de Ambiente
# Define as variáveis de ambiente para que o sistema operacional e o Python saibam onde encontrar as instalações do Java e do Spark.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

# 5. Instalação das bibliotecas Python para Spark
# Instala as bibliotecas Python necessárias para interagir com o Spark.
# !pip install -q: Instala os pacotes usando o gerenciador de pacotes do Python (pip) em modo silencioso.
#   pyspark==3.5.1: A biblioteca que fornece a API Python para o Spark. A versão é fixada para corresponder à versão do Spark baixada.
#   findspark: Uma biblioteca útil que ajuda o Python a localizar a instalação do Spark no sistema.
!pip install -q pyspark==3.5.1 findspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk-headless is already the newest version (11.0.27+6~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
spark-3.5.1-bin-hadoop3/
spark-3.5.1-bin-hadoop3/sbin/
spark-3.5.1-bin-hadoop3/sbin/spark-config.sh
spark-3.5.1-bin-hadoop3/sbin/stop-slave.sh
spark-3.5.1-bin-hadoop3/sbin/stop-mesos-dispatcher.sh
spark-3.5.1-bin-hadoop3/sbin/start-workers.sh
spark-3.5.1-bin-hadoop3/sbin/start-slaves.sh
spark-3.5.1-bin-hadoop3/sbin/start-all.sh
spark-3.5.1-bin-hadoop3/sbin/stop-all.sh
spark-3.5.1-bin-hadoop3/sbin/workers.sh
spark-3.5.1-bin-hadoop3/sbin/start-mesos-dispatcher.sh
spark-3.5.1-bin-hadoop3/sbin/spark-daemon.sh
spark-3.5.1-bin-hadoop3/sbin/decommission-worker.sh
s

In [2]:
from google.colab import drive, files
from pyspark.sql import SparkSession
import findspark
import requests
import gzip
import tarfile
import os
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import Row
from pyspark.sql.functions import col, count, sum, avg, when
findspark.init()



In [3]:
spark = SparkSession.builder \
    .appName("IFood Test Case") \
    .getOrCreate()

spark

In [4]:
###################################################################################
# SEÇÃO DE CONFIGURAÇÃO: ORIGEM DOS DADOS
# ---------------------------------------------------------------------------------
# Define se os arquivos de dados serão lidos a partir do Google Drive ou de uma
# pasta local (uploaded).
###################################################################################

# Variável de controle para a fonte dos dados.
# Opções válidas:
#   "drive": Para carregar os arquivos diretamente de uma pasta no Google Drive.
#   "uploaded": Para carregar os arquivos de um diretório local.
data_source = "drive"

# Define o caminho base no Google Drive onde os arquivos de dados estão localizados.
# Esta variável só é utilizada se 'data_source' for definida como "drive".
# Exemplo de estrutura de pastas: drive/MyDrive/iFood/Data/Bronze/
drive_silver_path = "drive/MyDrive/iFood/Data/Silver"
upload_silver_path = "./Data/Silver"  ##Pode variar caso os .parquets nao sejam exatamente os criados no primeiro notebook.

if data_source == "drive":
    drive.mount('/content/drive')
    silver_path = drive_silver_path
else:
    silver_path = upload_silver_path


Mounted at /content/drive


In [ ]:
### apenas para o fluxo usando upload manual...###########
######################################################

# Criar diretório local para salvar os arquivos
os.makedirs('Data', exist_ok=True)
# depois faca o upload dos zip nessa pasta, caso nao esteja usando o fluxo do google drive.


In [ ]:
### apenas para o fluxo usando upload manual...###########
######################################################

# Define uma variável para armazenar o nome da pasta onde os arquivos .zip
# devem ser encontrados. O caminho "./Data" refere-se a uma pasta chamada "Data"
# localizada no mesmo nível do diretório de execução do notebook (/content/).
caminho_da_pasta_zip = "./Data"


# Imprime uma mensagem informativa para o usuário saber qual ação está sendo executada.
print(f"\nProcurando arquivos .zip em: {caminho_da_pasta_zip}")

# Bloco de verificação para garantir que a pasta de origem existe.
# Se a pasta não for encontrada, o script exibe um erro claro em vez de falhar.
if not os.path.isdir(caminho_da_pasta_zip):
    print(f"ERRO: O diretório '{caminho_da_pasta_zip}' não foi encontrado.")
    print("Por favor, verifique o caminho e tente novamente.")
else:
    # Se a pasta existe, esta linha obtém uma lista de todos os nomes de arquivos e subpastas dentro dela.
    arquivos_na_pasta = os.listdir(caminho_da_pasta_zip)

    # Inicializa uma variável de controle para rastrear se algum arquivo .zip foi processado.
    zip_encontrado = False

    # Inicia um loop para examinar cada item encontrado na pasta de origem.
    for nome_arquivo in arquivos_na_pasta:
        # Condição para filtrar e processar apenas os itens que são arquivos .zip.
        if nome_arquivo.endswith(".zip"):
            # Atualiza a variável de controle, indicando que pelo menos um zip foi encontrado.
            zip_encontrado = True
            # Constrói o caminho completo para o arquivo .zip, unindo o nome da pasta e o nome do arquivo.
            caminho_completo_zip = os.path.join(caminho_da_pasta_zip, nome_arquivo)

            print(f"  -> Descompactando '{nome_arquivo}'...")

            # Utiliza um comando de shell (!unzip) para extrair o conteúdo do arquivo.
            # A opção "-o" força a substituição de arquivos existentes sem pedir permissão.
            # A opção "-d /content/" especifica o diretório de destino para a extração.
            !unzip -o "{caminho_completo_zip}" -d /content/

    # Após o loop, se a variável de controle não mudou, significa que nenhum zip foi processado.
    if not zip_encontrado:
        print("Nenhum arquivo .zip foi encontrado na pasta especificada.")

# Mensagem final para indicar que todas as operações foram concluídas.
print("\nProcesso de descompactação concluído! ✅")


# Bloco final para verificação visual dos resultados.
# O comando de shell "!ls" lista o conteúdo do diretório especificado.
print("\nConteúdo extraído no diretório (/content/Data/Bronze/):")
!ls -l /content/Data/Bronze

In [5]:
# Caminhos para os dados de pedidos
orders_silver_path = f"{silver_path}/orders.parquet"

# Caminhos para os dados de consumidores
consumers_silver_path = f"{silver_path}/consumers.parquet"

# Caminhos para os dados de restaurantes
restaurants_silver_path = f"{silver_path}/restaurants.parquet"

# Caminhos para os dados de referencia do teste A/B
ab_test_silver_path = f"{silver_path}/ab_test_ref.parquet"


In [6]:
dfp_orders = spark.read.parquet(orders_silver_path)
dfp_consumers = spark.read.parquet(consumers_silver_path)
dfp_restaurants = spark.read.parquet(restaurants_silver_path)
dfp_ab_test = spark.read.parquet(ab_test_silver_path)

In [ ]:
# --- Bloco de Verificação: Validação da Carga de Dados na Camada Silver ---
#
# O objetivo deste bloco é realizar um teste de sanidade (sanity check) para
# garantir que a ingestão de dados para a camada Silver ocorreu como esperado.
#

# Contagem de registros no DataFrame de pedidos (orders).
print("Contagem de registros em 'dfp_orders':")
print(dfp_orders.count())

# Contagem de registros no DataFrame de consumidores (consumers).
print("\nContagem de registros em 'dfp_consumers':")
print(dfp_consumers.count())

# Contagem de registros no DataFrame de restaurantes (restaurants).
print("\nContagem de registros em 'dfp_restaurants':")
print(dfp_restaurants.count())

# Contagem de registros no DataFrame de teste A/B (ab_test).
print("\nContagem de registros em 'dfp_ab_test':")
print(dfp_ab_test.count())

Contagem de registros em 'dfp_orders':
2426590

Contagem de registros em 'dfp_consumers':
804559

Contagem de registros em 'dfp_restaurants':
7292

Contagem de registros em 'dfp_ab_test':
804559


In [ ]:
print("--- Schema da Tabela: orders ---")
# Exibe a estrutura (colunas, tipos e nulidade) da tabela de pedidos.
dfp_orders.printSchema()

print("\n--- Schema da Tabela: consumers ---")
# Exibe a estrutura da tabela de clientes.
dfp_consumers.printSchema()

print("\n--- Schema da Tabela: restaurants ---")
# Exibe a estrutura da tabela de restaurantes.
dfp_restaurants.printSchema()

print("\n--- Schema da Tabela: ab_test ---")
# Exibe a estrutura da tabela de referência do teste A/B.
dfp_ab_test.printSchema()

--- Schema da Tabela: orders ---
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- order_created_at: string (nullable = true)
 |-- order_total_amount: double (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_latitude: string (nullable = true)
 |-- delivery_address_longitude: string (nullable = true)
 |-- origin_platform: string (nullable = true)
 |-- order_category: string (nullable = true)


--- Schema da Tabela: consumers ---
root
 |-- customer_id: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- customer_phone_state: string (nullable = true)


--- Schema da Tabela: restaurants ---
root
 |-- id: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- price_range: integer (nullable = true)
 |-- average_ticket: double (nullable = true)
 |-- delivery_time: intege

###DESAFIO 2####
A criação de segmentações permite agrupar usuários de acordo com
características e comportamentos similares, possibilitando criar estraté
gias direcionadas de acordo com o perfil de cada público, facilitando a
personalização e incentivando o engajamento, retenção, além de otimi
zação de recursos. Segmentações de usuários são muito utilizadas pe
los times de Data, mas a área em que você atua ainda não tem seg
mentos bem definidos e cada área de Negócio utiliza conceitos dife
rentes. Por isso, você precisa:

- a) Definir as segmentações que fazem sentido especificamente
para o teste A/B que está analisando.

- b) Estabelecer quais serão os critérios utilizados para cada seg
mento sugerido no item a). Utilize os critérios/ferramentas que
achar necessários, mas lembre-se de explicar o racional utiliza
do na criação.

- c) Analisar os resultados do teste A/B com base nos segmentos
definidos nos itens a) e b).

In [ ]:
from pyspark.sql.functions import to_date, col

# Converte a coluna 'created_at' para o tipo Date, caso ainda não seja.
# Assumimos que 'created_at' já foi limpa e está em um formato reconhecível (e.g., 'yyyy-MM-dd HH:mm:ss')
# Se a conversão para Parquet já tratou isso, esta linha pode ser redundante, mas é uma boa prática garantir.
dfp_consumers = dfp_consumers.withColumn("created_at_date", to_date(col("created_at")))

# Define a data de corte
cut_off_date = "2018-12-01"

# Filtra os consumidores criados antes de dezembro de 2018
consumers_before_dec_2018 = dfp_consumers.filter(col("created_at_date") < cut_off_date)

# Filtra os consumidores criados a partir de dezembro de 2018
consumers_from_dec_2018 = dfp_consumers.filter(col("created_at_date") >= cut_off_date)

# Conta o número de consumidores em cada grupo
count_before = consumers_before_dec_2018.count()
count_from = consumers_from_dec_2018.count()

# Imprime os resultados
print(f"Número de consumidores criados antes de Dezembro de 2018: {count_before}")
print(f"Número de consumidores criados a partir de Dezembro de 2018: {count_from}")

Número de consumidores criados antes de Dezembro de 2018: 804559
Número de consumidores criados a partir de Dezembro de 2018: 0


# Proposta de criar segmentação por tempo de vida é descartada pois para base que temos os valores sao homogeneos demais para gerar algum valor de analise.

## Desafio 2: Estratégia de Segmentação de Usuários

Com o impacto geral da campanha validado, o próximo passo é aprofundar a análise através da criação de segmentos de usuários. O objetivo é sair de uma abordagem de "um tamanho para todos" e entender como diferentes perfis de clientes reagiram à campanha de cupons. Isso nos permitirá otimizar futuras ações, tornando-as mais direcionadas, eficientes e lucrativas.

### O Framework de Segmentação Proposto

Para obter uma visão completa do comportamento do usuário, propomos uma abordagem de segmentação em duas camadas, que podem ser usadas em conjunto ou separadamente para criar "personas" detalhadas:

1.  **Segmentação por Padrão de Consumo (Dias da Semana):** Responde à pergunta "Qual o papel do iFood na vida do cliente?".
2.  **Segmentação por Valor e Engajamento (Modelo RFM):** Responde à pergunta "Qual o valor deste cliente para o negócio?".

A combinação das duas nos dará uma visão 360º de cada perfil de usuário.

### a) e b): Definição dos Segmentos e Seus Critérios

A seguir, detalhamos cada segmento proposto e os critérios exatos para sua criação, respondendo formalmente aos itens **2a e 2b**.

#### 2.1. Segmentação por Padrão de Consumo (Dias da Semana)

* **Racional:** Esta segmentação busca entender se o iFood é uma solução para a conveniência do dia a dia (almoço no trabalho, jantar durante a semana) ou uma ferramenta para momentos de lazer e socialização (pedidos de fim de semana). Cada um desses contextos apresenta oportunidades de marketing distintas.

* **Critérios (a serem calculados a partir dos dados de pedidos):**
    1.  Para cada pedido, extraímos o dia da semana.
    2.  Para cada cliente, calculamos a porcentagem de seus pedidos que ocorreram em "dias de semana" (Segunda a Quinta) vs. "fim de semana" (Sexta a Domingo).
    3.  Com base nessa proporção, criamos os seguintes segmentos:

| Segmento | Critério Sugerido | Perfil do Cliente e Comportamento |
| :--- | :--- | :--- |
| **Usuário de Fim de Semana** | > 70% dos pedidos ocorrem de Sexta a Domingo. | Vê o iFood como uma recompensa ou uma solução para encontros sociais. Provavelmente pede para mais de uma pessoa. |
| **Usuário de Dias de Semana** | > 70% dos pedidos ocorrem de Segunda a Quinta. | Vê o iFood como uma solução prática e funcional para a rotina. Pode ser mais sensível a preço e velocidade de entrega. |
| **Usuário Híbrido/Intensivo**| Pedidos distribuídos de forma equilibrada pela semana. | Perfil mais engajado. O iFood faz parte de toda a sua rotina, tanto para conveniência quanto para lazer. Provavelmente são os clientes de maior valor. |

#### 2.2. Segmentação por Valor e Engajamento (Modelo RFM)

* **Racional:** Este é o modelo padrão da indústria para quantificar o comportamento de compra e a "saúde" da conta de um cliente. Ele é altamente preditivo: clientes que compraram recentemente, com frequência e gastaram mais, têm a maior probabilidade de comprar novamente. O racional é focar os esforços de marketing de forma diferente em cada estrato de valor.

* **Critérios (calculados para cada cliente):**
    * **Recência (R):** Há quanto tempo o cliente fez seu último pedido. (Menor = Melhor)
    * **Frequência (F):** Com que frequência o cliente compra. (Maior = Melhor)
    * **Valor Monetário (M):** Quanto dinheiro o cliente gasta no total. (Maior = Melhor)

* **Criação dos Segmentos (Método de Pontuação):**
    1.  Calculamos R, F e M para cada cliente.
    2.  Dividimos os clientes em quartis (4 grupos) para cada métrica e atribuímos uma pontuação de 1 a 4 (onde 4 é o melhor comportamento).
    3.  Combinando as pontuações, criamos segmentos acionáveis como:

| Segmento | Pontuação RFM (Exemplo) | Perfil do Cliente | Ação Estratégica Sugerida |
| :--- | :--- | :--- | :--- |
| **Campeões** | 444 | Nossos melhores clientes: compram com frequência, recentemente e gastam muito. | Recompensar com ofertas exclusivas, não apenas descontos. |
| **Clientes Leais** | X4X (ex: 343, 442) | Compram com frequência, mas talvez não gastem tanto ou não compraram tão recentemente. | Reativar com bônus ou frete grátis para manter a lealdade. |
| **Em Risco** | 1XX (ex: 144, 134) | Eram bons clientes (alta F e M), mas não compram há muito tempo (baixa Recência). | **Alvo Perfeito para o cupom!** Precisam de um incentivo forte para retornarem. |
| **Hibernando** | 111, 212, etc. | Clientes com baixa frequência, baixo gasto e que não compram há muito tempo. | Campanha de reativação com desconto agressivo ou comunicação de baixo custo. |

O load para camada silver do df de orders ocorreu antes do que o df de consumers. Apesar de termos tratado isso no load de orders, podemos novamente ter casos de orders apontando para consumers que nao existem no df de consumers tratado.

In [ ]:
print("\n--- Bloco de Verificação: customer_id em dfp_orders existe em dfp_consumers ---")

# Coleta todos os customer_id únicos do DataFrame dfp_orders.
# Utiliza `collect()` para trazer os resultados para o driver,
# o que é adequado se a lista de IDs não for excessivamente grande.
# Caso contrário, seria melhor fazer a verificação no Spark.
order_customer_ids = set([row.customer_id for row in dfp_orders.select("customer_id").distinct().collect()])

# Coleta todos os customer_id únicos do DataFrame dfp_consumers.
consumer_ids = set([row.customer_id for row in dfp_consumers.select("customer_id").distinct().collect()])

# Encontra os customer_id que estão em dfp_orders mas não em dfp_consumers.
missing_customer_ids = order_customer_ids - consumer_ids

# Verifica se a lista de IDs faltantes está vazia.
if not missing_customer_ids:
    print("✅ Todos os customer_id referenciados em dfp_orders existem em dfp_consumers.")
else:
    print(f"❌ Encontrados {len(missing_customer_ids)} customer_id em dfp_orders que NÃO existem em dfp_consumers.")
    # Opcional: Imprimir alguns dos IDs faltantes para inspeção
    # print("Exemplos de customer_id faltantes:", list(missing_customer_ids)[:10])

print("--- Verificação concluída. ---")



--- Bloco de Verificação: customer_id em dfp_orders existe em dfp_consumers ---
❌ Encontrados 1597 customer_id em dfp_orders que NÃO existem em dfp_consumers.
--- Verificação concluída. ---


In [9]:
print("\n--- Filtrando dfp_orders para manter apenas customer_id válidos ---")

# Seleciona apenas os customer_id válidos da base de consumidores
valid_customers_df = dfp_consumers.select("customer_id").distinct()

# Realiza um inner join entre dfp_orders_cleaned e os customer_id válidos
dfp_orders_filtered = dfp_orders.join(
    valid_customers_df,
    on="customer_id",
    how="inner"
)

# Verificação de quantos registros foram removidos
original_count = dfp_orders.count()
filtered_count = dfp_orders_filtered.count()
removed_count = original_count - filtered_count

print(f"Filtragem concluída. {removed_count} registros com customer_id inválidos foram removidos.")
print(f"Total original: {original_count}")
print(f"Total após filtragem: {filtered_count}")



--- Filtrando dfp_orders para manter apenas customer_id válidos ---
Filtragem concluída. 5936 registros com customer_id inválidos foram removidos.
Total original: 2426590
Total após filtragem: 2420654


Df bases


In [10]:
# --- Bloco 1: Criação do DataFrame de Análise Base (dfp_analysis) ---
print("--- Bloco 1: Criando o DataFrame de Análise Base (do Desafio 1) ---")

# 1a. Contar pedidos por cliente
dfp_customer_orders = dfp_orders_filtered.groupBy("customer_id").agg(
    count("order_id").alias("order_count")
)

# 1b. Juntar com a base do teste A/B
dfp_analysis = dfp_ab_test.join(
    dfp_customer_orders,
    on="customer_id",
    how="inner" # Usamos 'inner' pois já validamos que todos no teste têm pedidos
)

# 1c. Criar a flag de retenção (nosso KPI primário)
dfp_analysis = dfp_analysis.withColumn(
    "is_retained",
    when(col("order_count") >= 2, True).otherwise(False)
)
print("DataFrame 'dfp_analysis' criado com sucesso.")

--- Bloco 1: Criando o DataFrame de Análise Base (do Desafio 1) ---
DataFrame 'dfp_analysis' criado com sucesso.


In [11]:
# --- Passo 0: Importações Necessárias ---
from pyspark.sql.functions import col, count, when, dayofweek, sum, lit, max, datediff
# Classes de Machine Learning para preparação dos dados e o teste estatístico
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.stat import ChiSquareTest
import math
import pandas as pd

Na mesma linha do que foi feito na limpeza para carregar a camada silver, os dados orfaos serao removidos

In [12]:
# --- Bloco 2: Criação da Segmentação por Padrão de Consumo ---
print("\n--- Bloco 2: Criando a Segmentação por Padrão de Consumo ---")

# 2a. Taguear Pedidos por Dia da Semana
dfp_orders_tagged = dfp_orders_filtered.withColumn(
    "day_of_week", dayofweek(col("order_created_at"))
).withColumn(
    "is_weekend",
    when(col("day_of_week").isin([1, 6, 7]), 1).otherwise(0) # 1=Dom, 6=Sex, 7=Sáb
)

# 2b. Calcular a Proporção de Pedidos por Cliente
dfp_customer_pattern = dfp_orders_tagged.groupBy("customer_id").agg(
    count("*").alias("total_orders"),
    sum("is_weekend").alias("weekend_orders")
)
dfp_customer_pattern = dfp_customer_pattern.withColumn(
    "weekend_proportion",
    col("weekend_orders") / col("total_orders")
)

# 2c. Atribuir os Segmentos Finais com base na proporção
WEEKEND_THRESHOLD = 0.69
WEEKDAY_THRESHOLD = 0.31
dfp_final_segmentation_pattern = dfp_customer_pattern.withColumn(
    "consumption_segment",
    when(col("weekend_proportion") > WEEKEND_THRESHOLD, lit("1"))
    .when(col("weekend_proportion") < WEEKDAY_THRESHOLD, lit("2"))
    .otherwise(lit("3"))
).select("customer_id", "consumption_segment")
print("DataFrame 'dfp_final_segmentation_pattern' criado com sucesso.")


--- Bloco 2: Criando a Segmentação por Padrão de Consumo ---
DataFrame 'dfp_final_segmentation_pattern' criado com sucesso.


In [13]:
# --- Bloco 3: Criação do DataFrame Master para Análise de Consumo ---
print("\n--- Bloco 3: Unindo os dados para criar o dfp_master_consumption ---")
dfp_master_consumption = dfp_analysis.join(
    dfp_final_segmentation_pattern, "customer_id", "inner"
)


--- Bloco 3: Unindo os dados para criar o dfp_master_consumption ---


In [14]:
# --- Passo 4: Verificar a Distribuição dos Segmentos ---
# É uma boa prática contar quantos clientes caíram em cada segmento.
print("\n--- Passo 4: Distribuição de Clientes por Segmento de Consumo ---")
dfp_final_segmentation_pattern.groupBy("consumption_segment").count().show()


--- Passo 4: Distribuição de Clientes por Segmento de Consumo ---
+-------------------+------+
|consumption_segment| count|
+-------------------+------+
|                  3|235943|
|                  1|212873|
|                  2|355743|
+-------------------+------+



In [ ]:
dfp_final_segmentation_pattern.show(5,truncate=False)

dfp_final_segmentation_pattern.select("customer_id").distinct().count()

+----------------------------------------------------------------+-------------------+
|customer_id                                                     |consumption_segment|
+----------------------------------------------------------------+-------------------+
|00006f567cb362ba98b0a23d9f9f73122e9ad98c9edb45bf2d5512068c2d1bf3|2                  |
|0000bb10fb47a1d6b2d73754ef383950ef536c77d07212e431c3ff77f68834c9|1                  |
|0000c21984ae00cefb5d4931bfa49483dde546413c9b40c4228220f27d7ecdf2|3                  |
|0001226e5175177581c0e520732ba58a61dfa96d57be0caef75f5bc948887a8c|1                  |
|0001274ea3bc24cee339c5bfe5c579c9175cb9a3a5050334d16ab68dcc5784aa|1                  |
+----------------------------------------------------------------+-------------------+
only showing top 5 rows



804559

In [ ]:
dfp_master_consumption.show(5,truncate=False)

+----------------------------------------------------------------+---------+-----------------------+--------------------+-----------+-----------+-------------------+
|customer_id                                                     |is_target|created_at             |customer_phone_state|order_count|is_retained|consumption_segment|
+----------------------------------------------------------------+---------+-----------------------+--------------------+-----------+-----------+-------------------+
|00006f567cb362ba98b0a23d9f9f73122e9ad98c9edb45bf2d5512068c2d1bf3|target   |2018-04-06 03:20:52.886|NaN                 |5          |true       |2                  |
|0000c21984ae00cefb5d4931bfa49483dde546413c9b40c4228220f27d7ecdf2|control  |2018-01-07 14:36:55.865|NaN                 |23         |true       |3                  |
|0001226e5175177581c0e520732ba58a61dfa96d57be0caef75f5bc948887a8c|target   |2018-03-31 23:13:40.808|RJ                  |2          |true       |1                  |
|000

In [ ]:
dfp_master_consumption.groupBy("is_target","consumption_segment","is_retained").count().orderBy("is_target", "consumption_segment", "is_retained").show()

+---------+-------------------+-----------+------+
|is_target|consumption_segment|is_retained| count|
+---------+-------------------+-----------+------+
|  control|                  1|      false| 78911|
|  control|                  1|       true| 22081|
|  control|                  2|      false|109493|
|  control|                  2|       true| 54670|
|  control|                  3|       true| 94544|
|   target|                  1|      false| 78729|
|   target|                  1|       true| 33152|
|   target|                  2|      false|109867|
|   target|                  2|       true| 81713|
|   target|                  3|       true|141399|
+---------+-------------------+-----------+------+



## Conclusão da Segmentação por Padrão de Consumo

A execução do código de segmentação por padrão de consumo foi bem-sucedida e nos forneceu a primeira camada de inteligência sobre a nossa base de usuários. A distribuição dos clientes nos três segmentos criados foi a seguinte:

* **Usuários de Dias de Semana:** ~355k clientes
* **Usuários Híbridos/Intensivos:** ~235k clientes
* **Usuários de Fim de Semana:** ~212k clientes

### Principais Insights da Análise

Esta divisão inicial já revela insights de negócio valiosos:

1.  **A Força do Hábito da Semana:** O maior segmento, de forma clara, é o de "Usuários de Dias de Semana". Isso sugere que, para uma grande parte da base, o iFood desempenha um papel fundamental como uma **solução de conveniência para a rotina**, como almoços durante o trabalho ou jantares práticos, e não apenas como um serviço para ocasiões de lazer.

2.  **O Valor do Usuário Intensivo:** O segundo maior grupo é o de "Usuários Híbridos/Intensivos". Este é um segmento extremamente importante, pois representa os clientes mais engajados, que integraram o iFood em todos os momentos de sua semana, indicando alta lealdade.

3.  **Desmistificando o "Foco no Fim de Semana":** Embora ainda seja um grupo grande, os "Usuários de Fim de Semana" representam o menor dos três segmentos. Isso desafia a possível percepção de que o iFood é uma plataforma predominantemente de lazer e indica a força do seu caso de uso prático no dia a dia.

> Essa clara distinção nos perfis de consumo é fundamental. Ela nos permitirá, na próxima etapa, analisar como a campanha de cupons impactou cada um desses grupos de forma diferente, abrindo caminho para estratégias de marketing muito mais personalizadas e eficientes.

In [15]:
# Mapeamento dos códigos para nomes legíveis
map_pattern = {'1': 'Usuário de Fim de Semana', '2': 'Usuário de Dias de Semana', '3': 'Usuário Híbrido/Intensivo'}

# Coletamos os códigos dos segmentos para iterar
segments_to_analyze = [row['consumption_segment'] for row in dfp_master_consumption.select("consumption_segment").distinct().collect()]

# Loop de Análise
for segment_code in segments_to_analyze:
    segment_name = map_pattern.get(segment_code, "Desconhecido")

    print("\n" + "="*50)
    print(f"### Iniciando Análise para o Segmento: {segment_name} ###")
    print("="*50)

    # Filtra o DataFrame para o segmento atual
    df_segment = dfp_master_consumption.filter(col("consumption_segment") == segment_code)

    # Prepara os dados para o ChiSquareTest do PySpark
    # 1. Converte a flag de retenção para inteiro (0 ou 1)
    df_segment = df_segment.withColumn("is_retained_int", col("is_retained").cast("integer"))

    # 2. Converte a coluna de grupo 'is_target' para um índice numérico 'label'
    label_indexer = StringIndexer(inputCol="is_target", outputCol="label")
    df_segment = label_indexer.fit(df_segment).transform(df_segment)

    # 3. Monta a coluna de feature em um formato de vetor
    feature_assembler = VectorAssembler(inputCols=["is_retained_int"], outputCol="features")
    dfp_prepared = feature_assembler.transform(df_segment)

    # --- Executando o Teste Qui-Quadrado ---
    # Usamos um try-except para o caso de um segmento não ter dados suficientes
    try:
        print(f"\n--- Teste Qui-Quadrado para o segmento '{segment_name}' ---")
        chi_square_result = ChiSquareTest.test(dfp_prepared, "features", "label").first()

        # Extraindo os resultados
        p_value = chi_square_result.pValues[0]
        chi2_statistic = chi_square_result.statistics[0]

        print(f"P-Valor do teste: {p_value:.8f}")
        print(f"Estatística Qui-Quadrado: {chi2_statistic:.4f}")

        # Veredito estatístico
        alpha = 0.05
        if p_value < alpha:
            print("Veredito: A diferença é ESTATISTICAMENTE SIGNIFICATIVA (Rejeitamos H0).")
        else:
            print("Veredito: A diferença NÃO é estatisticamente significativa (Não podemos rejeitar H0).")

    except Exception as e:
        print(f"AVISO: O teste não pôde ser executado para o segmento '{segment_name}'. Causa: {e}")


### Iniciando Análise para o Segmento: Usuário Híbrido/Intensivo ###

--- Teste Qui-Quadrado para o segmento 'Usuário Híbrido/Intensivo' ---
P-Valor do teste: 1.00000000
Estatística Qui-Quadrado: 0.0000
Veredito: A diferença NÃO é estatisticamente significativa (Não podemos rejeitar H0).

### Iniciando Análise para o Segmento: Usuário de Fim de Semana ###

--- Teste Qui-Quadrado para o segmento 'Usuário de Fim de Semana' ---
P-Valor do teste: 0.00000000
Estatística Qui-Quadrado: 1666.6616
Veredito: A diferença é ESTATISTICAMENTE SIGNIFICATIVA (Rejeitamos H0).

### Iniciando Análise para o Segmento: Usuário de Dias de Semana ###

--- Teste Qui-Quadrado para o segmento 'Usuário de Dias de Semana' ---
P-Valor do teste: 0.00000000
Estatística Qui-Quadrado: 3269.3172
Veredito: A diferença é ESTATISTICAMENTE SIGNIFICATIVA (Rejeitamos H0).


## Análise dos Resultados Estatísticos por Padrão de Consumo

A análise estatística do teste A/B para cada segmento de consumo revela um insight fundamental: **o impacto da campanha de cupons varia drasticamente dependendo do perfil de uso do cliente.** Os resultados mostram que a campanha foi eficaz para alguns grupos, mas completamente inócua para outros.

### Usuários de Fim de Semana e de Dias de Semana: Impacto Significativo

Para estes dois segmentos, a análise estatística confirma que a campanha funcionou.

* **Resultado:** Tanto para "Usuários de Fim de Semana" quanto para "Usuários de Dias de Semana", o p-valor foi `0.0000`, o que nos leva a **rejeitar a Hipótese Nula** com o mais alto grau de confiança.
* **Interpretação:** Isso significa que, para ambos os perfis, o cupom foi um **incentivo eficaz para aumentar a retenção**. Os clientes que concentram seus pedidos ou na rotina da semana ou nos momentos de lazer do fim de semana foram positivamente influenciados pelo desconto.
* **Observação Adicional:** É notável que a estatística Qui-Quadrado para o grupo de "Dias de Semana" (`3269`) é quase o dobro da do grupo de "Fim de Semana" (`1656.0`). Isso sugere que o efeito da campanha pode ter sido ainda mais forte e pronunciado nos usuários com perfil de consumo de rotina.

### Usuários Híbridos/Intensivos: Ausência de Impacto

O resultado para este segmento é o mais revelador da análise.

* **Resultado:** O p-valor de `1.0000` e a Estatística Qui-Quadrado de `0.0000` indicam que não houve **absolutamente nenhuma diferença** no comportamento de retenção entre os grupos controle e teste dentro deste segmento.
* **Interpretação:** O cupom foi **completamente ineficaz** para alterar o comportamento deste grupo. A explicação mais provável é que esses clientes, por já serem altamente engajados e utilizarem o iFood em todos os contextos (dias de semana e fins de semana), já possuem um hábito de consumo consolidado. Sua decisão de compra provavelmente é movida por necessidade e conveniência, e não por um incentivo pontual como um cupom.

### Implicações Estratégicas Iniciais

> A conclusão desta análise estatística é clara: o cupom não é uma ferramenta universal, mas sim uma alavanca de **mudança de comportamento**. Ele é poderoso para influenciar clientes com padrões definidos, mas ineficaz para aqueles que já atingiram um alto patamar de engajamento. Este é um insight crucial para a otimização de custos: oferecer este tipo de cupom para o segmento "Híbrido/Intensivo" com o objetivo de aumentar a retenção é um investimento com retorno nulo.

In [16]:
# --- Bloco 4: Análise de Impacto de Negócio por Segmento de Consumo ---

print("\n\n" + "="*80)
print("### ETAPA 2: ANÁLISE DE NEGÓCIO E IMPACTO POR PADRÃO DE CONSUMO ###")
print("="*80)

# Mapeamento dos códigos para nomes legíveis
map_pattern = {'1': 'Usuário de Fim de Semana', '2': 'Usuário de Dias de Semana', '3': 'Usuário Híbrido/Intensivo'}

# Coletamos os códigos dos segmentos para iterar
segments_to_analyze = [row['consumption_segment'] for row in dfp_master_consumption.select("consumption_segment").distinct().collect()]

# Lista para armazenar o resumo final
final_business_results = []

# Loop de Análise de Negócio
for segment_code in sorted(segments_to_analyze):
    segment_name = map_pattern.get(segment_code, "Desconhecido")

    print("\n" + "-"*60)
    print(f"### Calculando Métricas de Negócio para o Segmento: {segment_name} ###")
    print("-"*60)

    # Filtra o DataFrame para o segmento atual
    df_segment = dfp_master_consumption.filter(col("consumption_segment") == segment_code)

    # Agregação para obter os números da tabela de contingência
    summary_df = df_segment.groupBy("is_target").agg(
        count("*").alias("total_users"),
        count(when(col("is_retained") == True, True)).alias("retained_users")
    ).collect()

    # Verifica se temos ambos os grupos para comparar
    control_rows = [row for row in summary_df if row['is_target'] == 'control']
    target_rows = [row for row in summary_df if row['is_target'] == 'target']

    if not control_rows or not target_rows:
        print(f"AVISO: Dados insuficientes para o segmento '{segment_name}'. Análise pulada.")
        continue

    control_row = control_rows[0]
    target_row = target_rows[0]

    control_total = control_row['total_users']
    control_retained = control_row['retained_users']
    target_total = target_row['total_users']
    target_retained = target_row['retained_users']

    # Calcula as taxas de retenção
    retention_rate_control = control_retained / control_total if control_total > 0 else 0
    retention_rate_target = target_retained / target_total if target_total > 0 else 0

    print(f"Total de Usuários (Controle): {control_total}")
    print(f"Usuários Retidos (Controle): {control_retained}")
    print(f"Taxa de Retenção (Controle): {retention_rate_control:.2%}")
    print("-" * 30)
    print(f"Total de Usuários (Teste): {target_total}")
    print(f"Usuários Retidos (Teste): {target_retained}")
    print(f"Taxa de Retenção (Teste): {retention_rate_target:.2%}")

    # Calcula o Lift
    lift = ((retention_rate_target - retention_rate_control) / retention_rate_control) * 100 if retention_rate_control > 0 else 0
    print(f"\nLift Percentual (Aumento): {lift:.2f}%")

    # Calcula o Intervalo de Confiança para a diferença das proporções
    diff = retention_rate_target - retention_rate_control
    if target_total > 0 and control_total > 0:
        # Evita erro de divisão por zero no cálculo do erro padrão
        std_dev1_sq = retention_rate_target * (1 - retention_rate_target) / target_total
        std_dev2_sq = retention_rate_control * (1 - retention_rate_control) / control_total
        std_err_diff = math.sqrt(std_dev1_sq + std_dev2_sq)
        conf_interval_diff = (diff - 1.96 * std_err_diff, diff + 1.96 * std_err_diff)
    else:
        conf_interval_diff = (0, 0)

    print(f"Diferença Absoluta: {diff:.2%}")
    print(f"Intervalo de Confiança 95% para a Diferença: [{conf_interval_diff[0]:.2%}, {conf_interval_diff[1]:.2%}]")

    # Armazena o resultado para a tabela final
    final_business_results.append({
        'Segmento': segment_name,
        'Retenção (Controle)': retention_rate_control,
        'Retenção (Teste)': retention_rate_target,
        'Lift (%)': lift
    })

# --- Bloco 5: Apresentação da Tabela Resumo ---
print("\n\n" + "="*80)
print("### TABELA RESUMO: MÉTRICAS DE NEGÓCIO POR PADRÃO DE CONSUMO ###")
print("="*80)

df_final_biz = pd.DataFrame(final_business_results)

# Formatando as colunas
df_final_biz['Retenção (Controle)'] = df_final_biz['Retenção (Controle)'].apply(lambda x: f'{x:.2%}')
df_final_biz['Retenção (Teste)'] = df_final_biz['Retenção (Teste)'].apply(lambda x: f'{x:.2%}')
df_final_biz['Lift (%)'] = df_final_biz['Lift (%)'].apply(lambda x: f'{x:.2f}%')

# Ordenando por Lift para destacar os segmentos com maior impacto
print(df_final_biz.sort_values(by='Lift (%)', key=lambda x: x.str.replace('%', '').astype(float), ascending=False).to_string(index=False))





### ETAPA 2: ANÁLISE DE NEGÓCIO E IMPACTO POR PADRÃO DE CONSUMO ###

------------------------------------------------------------
### Calculando Métricas de Negócio para o Segmento: Usuário de Fim de Semana ###
------------------------------------------------------------
Total de Usuários (Controle): 100992
Usuários Retidos (Controle): 22081
Taxa de Retenção (Controle): 21.86%
------------------------------
Total de Usuários (Teste): 111881
Usuários Retidos (Teste): 33152
Taxa de Retenção (Teste): 29.63%

Lift Percentual (Aumento): 35.53%
Diferença Absoluta: 7.77%
Intervalo de Confiança 95% para a Diferença: [7.40%, 8.14%]

------------------------------------------------------------
### Calculando Métricas de Negócio para o Segmento: Usuário de Dias de Semana ###
------------------------------------------------------------
Total de Usuários (Controle): 164163
Usuários Retidos (Controle): 54670
Taxa de Retenção (Controle): 33.30%
------------------------------
Total de Usuários (Test

### Análise das Métricas de Negócio e Conclusão: Impacto por Padrão de Consumo

Com a verificação estatística conduzida para cada grupo segmentado por padrão de consumo, esta etapa quantifica o impacto **granular da campanha** nos diferentes perfis de usuários, oferecendo uma leitura mais direcionada e prática dos resultados.

#### Usuários de Fim de Semana

- **Taxa de Retenção (Controle):** 21.86%
- **Taxa de Retenção (Teste):** 29.63%
- **Lift Percentual:** +35.53%
- **Diferença Absoluta:** +7.77 pontos percentuais
- **Intervalo de Confiança 95%:** [7.40%, 8.14%]

Este é um resultado altamente expressivo. O grupo de usuários mais ativos nos fins de semana respondeu muito bem à campanha, com **ganhos consistentes e estatisticamente significativos**.

#### Usuários de Dias de Semana

- **Taxa de Retenção (Controle):** 33.30%
- **Taxa de Retenção (Teste):** 42.65%
- **Lift Percentual:** +28.08%
- **Diferença Absoluta:** +9.35 pontos percentuais
- **Intervalo de Confiança 95%:** [9.03%, 9.67%]

Os resultados para esse grupo também confirmam um **impacto positivo e robusto** da campanha, com ganhos sólidos tanto em termos absolutos quanto relativos, novamente com um intervalo de confiança inteiramente positivo.

#### Usuários Híbrido/Intensivo

- **Taxa de Retenção (Controle):** 100.00%
- **Taxa de Retenção (Teste):** 100.00%
- **Lift Percentual:** 0.00%
- **Diferença Absoluta:** 0.00 pontos percentuais
- **Intervalo de Confiança 95%:** [0.00%, 0.00%]

Este resultado, à primeira vista, sugere uma retenção perfeita para ambos os grupos — algo que, no contexto real, **não é plausível**. Essa anomalia pode ter sido causada por:

- Uma **distorção nos dados de origem** (ex: erro de processamento, sampleagem enviesada)
- Um problema nas **premissas de segregação dos grupos**
- Ou ainda uma definição equivocada de retenção para esse perfil específico

Em um ambiente de produção, esse segmento exigiria **investigação aprofundada** e revisão dos critérios de segmentação e cálculo. No contexto atual, optamos por **não considerar esse segmento** na conclusão final.

### Conclusão Geral por Segmento

> Com base nos dois primeiros segmentos — **Usuários de Fim de Semana** e **Usuários de Dias de Semana** — podemos afirmar com alta confiança que a campanha de cupons gerou **efeitos positivos, significativos e relevantes do ponto de vista de negócio**. O Lift Percentual em ambos os casos foi superior a 28%, com diferenças absolutas claras e estatisticamente sustentadas.
>
> Apesar da anomalia no grupo Híbrido/Intensivo, os demais resultados reforçam a conclusão de que a campanha **funciona de forma especialmente eficaz** para perfis de uso bem definidos, validando sua eficácia como alavanca de retenção segmentada.


Segmentação por Valor e Engajamento

In [17]:
# --- Passo 0: Importações Necessárias ---
from pyspark.sql.functions import col, count, sum, max, datediff, lit, concat, ntile, date_add
from pyspark.sql.window import Window


In [18]:
# --- Passo 1: Calcular R, F e M para cada Cliente ---
print("--- Passo 1: Calculando Recência, Frequência e Valor Monetário ---")

# A Recência precisa de um "hoje".
# vamos definir uma data de referência logo após o último pedido nos dados.
# Em um cenário real, usaríamos a data atual.
# Encontra a data máxima e já adiciona 1 dia, tudo dentro do Spark.
# A função date_add funciona com colunas do tipo Date ou Timestamp.
data_de_referencia_row = dfp_orders_filtered.agg(
    date_add(max("order_created_at"), 1).alias("ref_date")
).first()
data_de_referencia = data_de_referencia_row['ref_date']

print(f"Data de referência para cálculo da Recência (dinâmica): {data_de_referencia}")

dfp_rfm = dfp_orders_filtered.groupBy("customer_id").agg(
    # Recência (R): Diferença em dias da data de referência para o último pedido
    datediff(lit(data_de_referencia), max("order_created_at")).alias("recency"),
    # Frequência (F): Contagem total de pedidos
    count("order_id").alias("frequency"),
    # Monetário (M): Soma total do valor dos pedidos
    sum("order_total_amount").alias("monetary")
)

dfp_rfm.show(5)

--- Passo 1: Calculando Recência, Frequência e Valor Monetário ---
Data de referência para cálculo da Recência (dinâmica): 2019-02-01
+--------------------+-------+---------+-----------------+
|         customer_id|recency|frequency|         monetary|
+--------------------+-------+---------+-----------------+
|00006f567cb362ba9...|      8|        5|533.0999999999999|
|0000c21984ae00cef...|      1|       23|          1011.36|
|0001226e517517758...|     48|        2|             86.5|
|00021cd56b6d6c980...|     29|        3|             47.8|
|00021f6dc15d10418...|     16|        2|            174.6|
+--------------------+-------+---------+-----------------+
only showing top 5 rows



In [19]:
# --- Passo 2: Calcular os Scores RFM (1 a 4) ---
# Usamos a função ntile() sobre uma Window para criar os 4 quartis (grupos de 25%).

print("\n--- Passo 2: Criando os scores de R, F e M ---")

# A Window para Frequência e Monetário ordena do maior para o menor (quanto mais, melhor)
window_freq_monetary = Window.orderBy(col("value").desc())
# A Window para Recência ordena do menor para o maior (quanto menos dias, melhor)
window_recency = Window.orderBy(col("value").asc())

# Criando os scores
score_r = dfp_rfm.select("customer_id", col("recency").alias("value")) \
    .withColumn("r_score", ntile(4).over(window_recency)) \
    .select("customer_id", "r_score")

score_f = dfp_rfm.select("customer_id", col("frequency").alias("value")) \
    .withColumn("f_score", ntile(4).over(window_freq_monetary)) \
    .select("customer_id", "f_score")

score_m = dfp_rfm.select("customer_id", col("monetary").alias("value")) \
    .withColumn("m_score", ntile(4).over(window_freq_monetary)) \
    .select("customer_id", "m_score")

# Juntando os scores em um único DataFrame
dfp_rfm_scores = dfp_rfm.join(score_r, "customer_id") \
                        .join(score_f, "customer_id") \
                        .join(score_m, "customer_id")

dfp_rfm_scores.select("customer_id", "recency", "r_score", "frequency", "f_score", "monetary", "m_score").show(10)



--- Passo 2: Criando os scores de R, F e M ---
+--------------------+-------+-------+---------+-------+--------+-------+
|         customer_id|recency|r_score|frequency|f_score|monetary|m_score|
+--------------------+-------+-------+---------+-------+--------+-------+
|0001226e517517758...|     48|      4|        2|      2|    86.5|      2|
|00021cd56b6d6c980...|     29|      3|        3|      1|    47.8|      3|
|00021f6dc15d10418...|     16|      2|        2|      2|   174.6|      1|
|00022b8c0c7af061f...|     19|      2|        5|      1|    80.9|      2|
|00029b26fb2121119...|     10|      2|        1|      3|    54.6|      3|
|0002cc7394d677fdf...|     49|      4|        1|      3|    43.9|      3|
|0003fc1a7fe21c5f4...|      6|      1|        2|      2|    69.0|      3|
|000405bb6de6550fe...|      3|      1|        3|      1|  175.07|      1|
|0005bc77d13c12d94...|     18|      2|        5|      1|   315.9|      1|
|0006aba7fd94d9de3...|     45|      4|        2|      2|   109.9

In [20]:
# --- Passo 3: Criar o Score RFM e os Segmentos Finais ---
print("\n--- Passo 3: Criando os segmentos RFM finais ---")

# Concatenamos os scores para criar um score RFM final (ex: "444")
dfp_final_segmentation_rfm = dfp_rfm_scores.withColumn(
    "rfm_score",
    concat(col("r_score"), col("f_score"), col("m_score"))
)

# Mapeamos os scores para os segmentos de negócio que definimos
dfp_final_segmentation_rfm = dfp_final_segmentation_rfm.withColumn(
    "rfm_segment",
    when(col("rfm_score") == "444", lit("1"))
    .when((col("f_score") == 4) & (col("r_score") >= 3), lit("2"))
    .when(col("r_score") <= 2, lit("3")) # Simplificação de usuários que não compram há tempos
    .when((col("r_score") >= 3) & (col("f_score") <= 2), lit("4"))
    .otherwise(lit("5"))
)

#grupo campeao = 1
#grupo Clientes LEais = 2
#grupo "Em Risco / Hibernando = 3
#grupo Novos / Potenciais = 4
#grupo Outros = 5

# Vamos selecionar as colunas finais e ver o resultado
dfp_final_segmentation_rfm = dfp_final_segmentation_rfm.select("customer_id", "rfm_segment", "rfm_score")
dfp_final_segmentation_rfm.show(10)


--- Passo 3: Criando os segmentos RFM finais ---
+--------------------+-----------+---------+
|         customer_id|rfm_segment|rfm_score|
+--------------------+-----------+---------+
|00006f567cb362ba9...|          3|      211|
|0000c21984ae00cef...|          3|      111|
|0001226e517517758...|          4|      422|
|00021cd56b6d6c980...|          4|      313|
|00021f6dc15d10418...|          3|      221|
|0002287b123ac1afc...|          5|      433|
|00022b8c0c7af061f...|          3|      212|
|00027035d16a4de43...|          5|      332|
|00029b26fb2121119...|          3|      233|
|0002cc7394d677fdf...|          5|      433|
+--------------------+-----------+---------+
only showing top 10 rows



In [ ]:
# --- Passo 4: Verificar a Distribuição dos Segmentos ---
print("\n--- Passo 4: Distribuição de Clientes por Segmento RFM ---")
dfp_final_segmentation_rfm.groupBy("rfm_segment").count().orderBy(col("count").desc()).show()



--- Passo 4: Distribuição de Clientes por Segmento RFM ---
+-----------+------+
|rfm_segment| count|
+-----------+------+
|          3|402280|
|          5|137219|
|          4|122712|
|          2|100628|
|          1| 41720|
+-----------+------+



## Conclusão da Segmentação por Valor e Engajamento (RFM)

A aplicação do modelo RFM (Recência, Frequência, Valor Monetário) nos permitiu classificar a base de clientes em segmentos de alto valor estratégico. Esta é a segunda camada da nossa análise, focada em entender o valor e o nível de engajamento de cada usuário.

### Recapitulação da Metodologia e Regras Aplicadas

Para construir os segmentos, seguimos um processo de 3 passos:

1.  **Cálculo das Métricas Individuais:** Para cada cliente, calculamos:
    * **Recência (R):** O número de dias entre o último pedido do cliente e a data de referência final do nosso dataset.
    * **Frequência (F):** A contagem total de pedidos realizados pelo cliente no período.
    * **Valor Monetário (M):** A soma total (em R$) de todos os pedidos realizados pelo cliente.

2.  **Atribuição de Scores:** Cada uma dessas três métricas foi dividida em quartis (4 grupos de 25% do total de usuários). Com base nisso, cada cliente recebeu uma pontuação de 1 a 4 para cada métrica (R, F e M). A pontuação 4 representa o melhor quartil (ex: clientes mais recentes, com maior frequência ou maior gasto).

3.  **Mapeamento para Segmentos de Negócio:** Os scores individuais foram combinados para formar um "RFM Score" (ex: "444") e, em seguida, mapeados para segmentos de negócio acionáveis, conforme a regra que definimos.

### Distribuição dos Clientes e Insights Estratégicos

A distribuição dos clientes nos segmentos criados foi a seguinte:

* **Em Risco / Hibernando:** 402280 clientes (~50.0%)
* **Outros:** 137219 clientes (~17.0%)
* **Novos / Potenciais:** 122712 clientes (~15.3%)
* **Clientes Leais:** 100628 clientes (~12.5%)
* **Campeões:** 41720 clientes (~5.2%)

#### Análise Crítica dos Resultados

Esta distribuição revela um panorama claro da base de clientes do iFood neste período:

1.  **A Grande Oportunidade de Reativação:** O insight mais impactante é que o maior segmento, correspondendo a **metade da base de clientes**, é o de usuários "Em Risco / Hibernando". Trata-se de um grupo massivo de clientes que já demonstraram valor, mas que não compram há tempos. Eles representam a maior oportunidade de crescimento incremental, pois uma campanha de cupons bem direcionada tem um potencial enorme para "despertá-los".

2.  **O Valioso Núcleo de "Campeões":** Como é típico em muitos negócios, um pequeno grupo de clientes (~5%) representa a elite de usuários. Os "Campeões" são o ativo mais valioso. A estratégia para eles não deve ser a de oferecer descontos genéricos (que podem canibalizar uma receita já garantida), mas sim a de criar um relacionamento de exclusividade e recompensa pela lealdade.

3.  **A Base de Crescimento:** Os segmentos de "Clientes Leais" e "Novos / Potenciais" formam a espinha dorsal da empresa. São clientes ativos que constituem o "celeiro" de onde surgirão os futuros campeões. As estratégias para eles devem focar em aumentar a frequência e o ticket médio para movê-los para o topo da pirâmide de valor.



In [21]:

# --- Bloco 1: Preparação para a Análise Segmentada ---
print("--- Bloco 1: Criando o DataFrame de Análise por Segmento RFM ---")
dfp_master_rfm = dfp_analysis.join(
    dfp_final_segmentation_rfm, "customer_id", "inner"
)

print("Master DataFrame para análise RFM pronto.")
dfp_master_rfm.show(5)

--- Bloco 1: Criando o DataFrame de Análise por Segmento RFM ---
Master DataFrame para análise RFM pronto.
+--------------------+---------+--------------------+--------------------+-----------+-----------+-----------+---------+
|         customer_id|is_target|          created_at|customer_phone_state|order_count|is_retained|rfm_segment|rfm_score|
+--------------------+---------+--------------------+--------------------+-----------+-----------+-----------+---------+
|0001226e517517758...|   target|2018-03-31 23:13:...|                  RJ|          2|       true|          4|      422|
|00021cd56b6d6c980...|   target|2018-01-04 15:16:...|                  PE|          3|       true|          4|      313|
|00021f6dc15d10418...|   target|2018-01-30 21:16:...|                  RS|          2|       true|          3|      221|
|00022b8c0c7af061f...|  control|2018-04-06 02:04:...|                  PB|          5|       true|          3|      212|
|00029b26fb2121119...|   target|2018-01-03 23:

In [22]:
# --- Bloco 2: Análise Estatística para Cada Segmento RFM ---
# Este bloco itera sobre cada segmento, executa o teste estatístico,
# e salva os resultados para apresentar em uma tabela resumo no final.

# Pré-requisitos:
# - dfp_master_rfm: DataFrame com os dados de teste e a coluna 'rfm_segment'
# - Imports: StringIndexer, VectorAssembler, ChiSquareTest, col

print("\n\n" + "="*80)
print("### ETAPA 1: ANÁLISE ESTATÍSTICA POR SEGMENTO RFM ###")
print("="*80)

# Mapeamento dos códigos para nomes legíveis
map_rfm = {'1': 'Campeões', '2': 'Clientes Leais', '3': 'Em Risco / Hibernando', '4': 'Novos / Potenciais', '5': 'Outros'}

# Coletamos os códigos dos segmentos para iterar
segments_to_analyze = [row['rfm_segment'] for row in dfp_master_rfm.select("rfm_segment").distinct().collect()]

# Lista para armazenar o resumo final dos resultados estatísticos
statistical_results_rfm = []

# Loop de Análise
for segment_code in sorted(segments_to_analyze):
    segment_name = map_rfm.get(segment_code, "Desconhecido")

    print(f"Processando o segmento: '{segment_name}'...")

    # Filtra o DataFrame para o segmento atual
    df_segment = dfp_master_rfm.filter(col("rfm_segment") == segment_code)

    # Prepara os dados para o ChiSquareTest do PySpark
    df_segment = df_segment.withColumn("is_retained_int", col("is_retained").cast("integer"))
    label_indexer = StringIndexer(inputCol="is_target", outputCol="label")
    df_segment = label_indexer.fit(df_segment).transform(df_segment)
    feature_assembler = VectorAssembler(inputCols=["is_retained_int"], outputCol="features")
    dfp_prepared = feature_assembler.transform(df_segment)

    # --- Executando o Teste Qui-Quadrado ---
    try:
        chi_square_result = ChiSquareTest.test(dfp_prepared, "features", "label").first()

        p_value = chi_square_result.pValues[0]
        chi2_statistic = chi_square_result.statistics[0]

        alpha = 0.05
        verdict = "SIGNIFICATIVA" if p_value < alpha else "NÃO SIGNIFICATIVA"

        # Armazena o resultado em vez de imprimir imediatamente
        statistical_results_rfm.append({
            'Segmento': segment_name,
            'P-Valor': p_value,
            'Estatística X²': chi2_statistic,
            'Veredito (alpha=0.05)': verdict
        })

    except Exception as e:
        print(f"AVISO: O teste não pôde ser executado para o segmento '{segment_name}'. Causa: {e}")
        # Armazena um resultado de erro
        statistical_results_rfm.append({
            'Segmento': segment_name,
            'P-Valor': np.nan,
            'Estatística X²': np.nan,
            'Veredito (alpha=0.05)': 'ERRO NO CÁLCULO'
        })

# --- Apresentação da Tabela Resumo dos Resultados Estatísticos ---
print("\n" + "="*80)
print("### TABELA RESUMO: RESULTADOS ESTATÍSTICOS POR SEGMENTO RFM ###")
print("="*80)

df_final_stats = pd.DataFrame(statistical_results_rfm)

# Formatando as colunas para melhor leitura
df_final_stats['P-Valor'] = df_final_stats['P-Valor'].apply(lambda x: f'{x:.4f}' if pd.notna(x) else 'N/A')
df_final_stats['Estatística X²'] = df_final_stats['Estatística X²'].apply(lambda x: f'{x:.2f}' if pd.notna(x) else 'N/A')

print(df_final_stats.to_string(index=False))



### ETAPA 1: ANÁLISE ESTATÍSTICA POR SEGMENTO RFM ###
Processando o segmento: 'Campeões'...
Processando o segmento: 'Clientes Leais'...
Processando o segmento: 'Em Risco / Hibernando'...
Processando o segmento: 'Novos / Potenciais'...
Processando o segmento: 'Outros'...

### TABELA RESUMO: RESULTADOS ESTATÍSTICOS POR SEGMENTO RFM ###
             Segmento P-Valor Estatística X² Veredito (alpha=0.05)
             Campeões  1.0000           0.00     NÃO SIGNIFICATIVA
       Clientes Leais  1.0000           0.00     NÃO SIGNIFICATIVA
Em Risco / Hibernando  0.0000        3202.24         SIGNIFICATIVA
   Novos / Potenciais  1.0000           0.00     NÃO SIGNIFICATIVA
               Outros  0.0000         431.75         SIGNIFICATIVA


## Análise dos Resultados Estatísticos por Segmento RFM

A análise estatística do teste A/B para cada segmento RFM revela uma dicotomia clara e poderosa: a campanha de cupons teve um **impacto massivo em clientes de baixo engajamento** e um **impacto nulo em clientes de alto engajamento**. Este é o insight mais estratégico da nossa análise de segmentação.

### Segmentos de Baixo Engajamento ("Em Risco" e "Outros"): Sucesso Absoluto na Reativação

* **Resultado:** Os segmentos **"Em Risco / Hibernando"** e **"Outros"** apresentaram p-valores de `0.0000`, levando a uma rejeição inequívoca da Hipótese Nula. A diferença no comportamento de retenção para estes grupos foi **altamente significativa**.
* **Interpretação:** O cupom funcionou como uma ferramenta de **reativação extremamente eficaz**. Para clientes que não compravam há muito tempo ou que tinham um padrão de compra indefinido, o desconto foi o gatilho necessário para trazê-los de volta à plataforma. A estatística Qui-Quadrado massiva para o grupo "Em Risco" (`3202`) indica que o efeito foi mais forte neste grupo.

### Segmentos de Alto Engajamento ("Campeões" e "Clientes Leais"): Efeito Nulo

* **Resultado:** Os segmentos **"Campeões"** e **"Clientes Leais"** apresentaram p-valores de `1.0000` e estatísticas Qui-Quadrado de `0.0000`. O veredito é que a diferença foi **estatisticamente não significativa**.
* **Interpretação:** O resultado de "efeito zero" é um insight crucial. Ele nos diz que o comportamento de compra desses clientes de alto valor **não é influenciado por este tipo de cupom**. Eles já são leais e sua frequência de compra é ditada por hábito e necessidade, não por um desconto pontual. Oferecer este cupom para eles representa uma **canibalização de receita**, ou seja, um custo desnecessário em uma compra que já iria acontecer.

### Segmento "Novos / Potenciais": Um Insight sobre Ativação

* **Resultado:** Assim como os clientes de alto valor, este segmento também mostrou um **efeito nulo** (p-valor de `1.0000`).
* **Interpretação:** Isso sugere que, para um usuário que ainda está nas fases iniciais de sua jornada, um cupom de retenção pode não ser o principal motivador para uma segunda compra imediata. Fatores como a qualidade da primeira experiência (entrega, restaurante, produto) podem ter um peso muito maior na decisão de se tornar um cliente recorrente.

### Implicações Estratégicas Imediatas

> A segmentação RFM nos forneceu um manual de instruções claro para a estratégia de cuponagem do iFood. A campanha não deve ser uma ação de massa, mas sim uma ferramenta cirúrgica:
>
> 1.  **Foco Total em Reativação:** Os cupons mais agressivos devem ser direcionados ao segmento **"Em Risco / Hibernando"**, onde o ROI é potencialmente maior.
> 2.  **Mudar a Estratégia para Clientes Topo de Funil:** Para os **"Campeões"** e **"Clientes Leais"**, a estratégia deve migrar de "descontos" para "recompensas e exclusividade" (programas de lealdade, benefícios não-financeiros).


In [23]:
# --- Bloco 5: Análise de Impacto de Negócio por Segmento RFM ---

print("\n\n" + "="*80)
print("### ETAPA 2: ANÁLISE DE NEGÓCIO E IMPACTO POR SEGMENTO RFM ###")
print("="*80)

# Mapeamento dos códigos para nomes legíveis
map_rfm = {'1': 'Campeões', '2': 'Clientes Leais', '3': 'Em Risco / Hibernando', '4': 'Novos / Potenciais', '5': 'Outros'}

# Coletamos os códigos dos segmentos para iterar
segments_to_analyze = [row['rfm_segment'] for row in dfp_master_rfm.select("rfm_segment").distinct().collect()]

# Lista para armazenar o resumo final
final_business_results_rfm = []

# Loop de Análise de Negócio
for segment_code in sorted(segments_to_analyze):
    segment_name = map_rfm.get(segment_code, "Desconhecido")

    print("\n" + "-"*60)
    print(f"### Calculando Métricas de Negócio para o Segmento: {segment_name} ###")
    print("-"*60)

    # Filtra o DataFrame para o segmento atual
    df_segment = dfp_master_rfm.filter(col("rfm_segment") == segment_code)

    # Agregação para obter os números da tabela de contingência
    summary_df = df_segment.groupBy("is_target").agg(
        count("*").alias("total_users"),
        count(when(col("is_retained") == True, True)).alias("retained_users")
    ).collect()

    # Verifica se temos ambos os grupos para comparar
    control_rows = [row for row in summary_df if row['is_target'] == 'control']
    target_rows = [row for row in summary_df if row['is_target'] == 'target']

    if not control_rows or not target_rows:
        print(f"AVISO: Dados insuficientes para o segmento '{segment_name}'. Análise pulada.")
        continue

    control_row = control_rows[0]
    target_row = target_rows[0]

    control_total = control_row['total_users']
    control_retained = control_row['retained_users']
    target_total = target_row['total_users']
    target_retained = target_row['retained_users']

    # Calcula as taxas de retenção
    retention_rate_control = control_retained / control_total if control_total > 0 else 0
    retention_rate_target = target_retained / target_total if target_total > 0 else 0

    print(f"Total de Usuários (Controle): {control_total}")
    print(f"Usuários Retidos (Controle): {control_retained}")
    print(f"Taxa de Retenção (Controle): {retention_rate_control:.2%}")
    print("-" * 30)
    print(f"Total de Usuários (Teste): {target_total}")
    print(f"Usuários Retidos (Teste): {target_retained}")
    print(f"Taxa de Retenção (Teste): {retention_rate_target:.2%}")

    # Calcula o Lift
    lift = ((retention_rate_target - retention_rate_control) / retention_rate_control) * 100 if retention_rate_control > 0 else 0
    print(f"\nLift Percentual (Aumento): {lift:.2f}%")

    # Calcula o Intervalo de Confiança para a diferença das proporções
    diff = retention_rate_target - retention_rate_control
    if target_total > 0 and control_total > 0:
        std_dev1_sq = retention_rate_target * (1 - retention_rate_target) / target_total
        std_dev2_sq = retention_rate_control * (1 - retention_rate_control) / control_total
        std_err_diff = math.sqrt(std_dev1_sq + std_dev2_sq)
        conf_interval_diff = (diff - 1.96 * std_err_diff, diff + 1.96 * std_err_diff)
    else:
        conf_interval_diff = (0, 0)

    print(f"Diferença Absoluta: {diff:.2%}")
    print(f"Intervalo de Confiança 95% para a Diferença: [{conf_interval_diff[0]:.2%}, {conf_interval_diff[1]:.2%}]")

    # Armazena o resultado para a tabela final
    final_business_results_rfm.append({
        'Segmento': segment_name,
        'Retenção (Controle)': retention_rate_control,
        'Retenção (Teste)': retention_rate_target,
        'Lift (%)': lift
    })

# --- Bloco Final: Apresentação da Tabela Resumo ---
print("\n\n" + "="*80)
print("### TABELA RESUMO: MÉTRICAS DE NEGÓCIO POR SEGMENTO RFM ###")
print("="*80)

df_final_biz_rfm = pd.DataFrame(final_business_results_rfm)

# Formatando as colunas
df_final_biz_rfm['Retenção (Controle)'] = df_final_biz_rfm['Retenção (Controle)'].apply(lambda x: f'{x:.2%}')
df_final_biz_rfm['Retenção (Teste)'] = df_final_biz_rfm['Retenção (Teste)'].apply(lambda x: f'{x:.2%}')
df_final_biz_rfm['Lift (%)'] = df_final_biz_rfm['Lift (%)'].apply(lambda x: f'{x:.2f}%')

# Ordenando por Lift para destacar os segmentos com maior impacto
print(df_final_biz_rfm.sort_values(by='Lift (%)', key=lambda x: x.str.replace('%', '').astype(float), ascending=False).to_string(index=False))




### ETAPA 2: ANÁLISE DE NEGÓCIO E IMPACTO POR SEGMENTO RFM ###

------------------------------------------------------------
### Calculando Métricas de Negócio para o Segmento: Campeões ###
------------------------------------------------------------
Total de Usuários (Controle): 20917
Usuários Retidos (Controle): 0
Taxa de Retenção (Controle): 0.00%
------------------------------
Total de Usuários (Teste): 20803
Usuários Retidos (Teste): 0
Taxa de Retenção (Teste): 0.00%

Lift Percentual (Aumento): 0.00%
Diferença Absoluta: 0.00%
Intervalo de Confiança 95% para a Diferença: [0.00%, 0.00%]

------------------------------------------------------------
### Calculando Métricas de Negócio para o Segmento: Clientes Leais ###
------------------------------------------------------------
Total de Usuários (Controle): 50201
Usuários Retidos (Controle): 0
Taxa de Retenção (Controle): 0.00%
------------------------------
Total de Usuários (Teste): 50427
Usuários Retidos (Teste): 0
Taxa de Reten

## Análise das Métricas de Negócio e Conclusão: Impacto por Segmento RFM

Com base na análise estatística e nos indicadores de negócio extraídos por segmentação RFM, é possível obter insights relevantes sobre o desempenho da campanha de retenção, considerando o comportamento passado dos usuários em relação à Recência, Frequência e Valor Monetário.

---

### Segmento: Em Risco / Hibernando
- **Taxa de Retenção (Controle):** 67.93%  
- **Taxa de Retenção (Teste):** 75.98%  
- **Lift Percentual:** +11.85%  
- **Diferença Absoluta:** +8.05 pontos percentuais  
- **Intervalo de Confiança 95%:** [7.77%, 8.33%]

Este é um dos segmentos mais estratégicos da base: usuários que já tiveram histórico positivo, mas apresentam risco de abandono ou já se encontram inativos. A campanha mostrou excelente desempenho nesse grupo, conseguindo reengajar de forma significativa. Trata-se de um resultado acionável e diretamente aplicável para estratégias de winback e remarketing. Uma abordagem personalizada para esse grupo pode consolidar essa reversão de tendência.

---

### Segmento: Outros
- **Taxa de Retenção (Controle):** 7.85%  
- **Taxa de Retenção (Teste):** 11.14%  
- **Lift Percentual:** +41.98%  
- **Diferença Absoluta:** +3.29 pontos percentuais  
- **Intervalo de Confiança 95%:** [2.98%, 3.60%]

Apesar da baixa taxa de retenção inicial, o grupo "Outros" apresentou o maior **lift relativo**. Esse segmento possivelmente concentra usuários com comportamentos não suficientemente consistentes para serem classificados em grupos tradicionais — ou seja, pode representar **clientes indecisos, novos em fase de experimentação ou com baixa frequência**. A campanha mostrou-se eficiente como uma possível “primeira âncora” para introduzir o app na rotina desses clientes. Estratégias de onboarding e nutrição automatizada podem transformar esses contatos em perfis mais fiéis ao longo do tempo.

---

### Segmentos com 0% e 100% de Retenção

#### Campeões e Clientes Leais
- Ambos os grupos apresentaram **0% de retenção** para controle e teste.
- Isso indica possível inconsistência ou ausência de eventos de retenção registrados no período de análise para esses segmentos.
- Pode estar relacionado a filtros de tempo, definição de métrica de retenção ou mesmo a uma **base de dados incompleta** para usuários que, por padrão, não estavam expostos ao risco de churn.

#### Novos / Potenciais
- Apresentaram **100% de retenção** tanto no grupo de controle quanto no de teste.
- Embora esse número pareça excelente, ele é irreal em contextos operacionais. Provavelmente estamos diante de:
  - Uma definição de retenção inadequada para o ciclo de vida curto desse grupo;
  - Ou uma limitação nos dados de observação no tempo (ex: usuários muito recentes).
- Recomendamos investigar a janela de tempo utilizada ou redefinir a métrica de retenção para este tipo de perfil.

---

## Conclusão Geral

Os resultados apontam para uma eficácia comprovada da campanha de retenção em dois segmentos centrais:

1. **"Em Risco / Hibernando"** — usuários com forte histórico que estavam se distanciando. A campanha teve efeito expressivo de reativação.
2. **"Outros"** — clientes indefinidos, com resposta positiva ao estímulo, indicando potencial para nutrição e fidelização.

Já os segmentos com 0% ou 100% de retenção devem ser tratados com cautela. São importantes para entender limitações dos dados e das definições de métrica de sucesso, podendo indicar necessidade de ajustes na modelagem ou observação de tempo.

Assim, as evidências reforçam que a segmentação RFM é um critério eficaz para ações de retenção, com resultados acionáveis em perfis estratégicos e oportunidades claras de otimização.


## Introdução: Análise Final de Impacto por Segmentos Combinados (Consumo + RFM)

Com base nas análises estatísticas e de negócio realizadas separadamente por **padrão de consumo** e por **perfil RFM**, foi possível identificar dois pontos de atenção estratégicos:

1. **Usuários "Em Risco / Hibernando"** apresentaram forte resposta à campanha, com melhoria estatisticamente significativa na taxa de retenção, validando o potencial de ações de reengajamento para esse público.
2. **Usuários classificados como "Outros"** — embora com baixa retenção inicial — mostraram um dos maiores *lifts percentuais* após a campanha. Esse grupo pode representar clientes em fase de descoberta ou uso esporádico, e a campanha parece ter funcionado como um ponto de entrada eficaz.

A partir desses achados, esta etapa propõe uma **análise combinada entre os dois eixos de segmentação**: comportamento de consumo e estágio de relacionamento (RFM). O objetivo é identificar interseções que potencializem a ação de retenção, ampliando o entendimento sobre **quais perfis compostos respondem melhor à campanha**.

Serão avaliadas as seguintes combinações:
- **Usuários de Fim de Semana + Em Risco / Hibernando**
- **Usuários de Dias de Semana + Em Risco / Hibernando**
- **Usuários de Fim de Semana + Outros**
- **Usuários de Dias de Semana + Outros**

A análise considera tanto a **significância estatística** (via teste Qui-Quadrado), quanto **indicadores de negócio** como taxa de retenção, lift percentual e intervalo de confiança da diferença absoluta.

Este aprofundamento visa apoiar decisões de personalização e segmentação mais refinadas, oferecendo **insights acionáveis para estratégias de retenção direcionadas a perfis compostos específicos**.


In [24]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.stat import ChiSquareTest
from pyspark.sql.functions import *

import math

print("\n\n" + "="*100)
print("### ANÁLISE FINAL: SEGMENTOS COMBINADOS (PADRÃO DE CONSUMO + RFM) ###")
print("="*100)

# Mapeamentos legíveis
map_consumo = {'1': 'Usuário de Fim de Semana', '2': 'Usuário de Dias de Semana'}
map_rfm = {'3': 'Em Risco / Hibernando','5': 'Outros'}

# Unimos as segmentações para criar um DataFrame combinado
dfp_full = dfp_analysis \
    .join(dfp_final_segmentation_pattern, "customer_id", "inner") \
    .join(dfp_final_segmentation_rfm, "customer_id", "inner")

# Lista para guardar os resultados
final_cross_results = []

# Itera sobre as combinações desejadas
for consumo_code in ['1', '2']:  # Fim de Semana e Dias de Semana
    for rfm_code in ['3','5']:       #  "Em Risco" e Outros
        consumo_label = map_consumo.get(consumo_code)
        rfm_label = map_rfm.get(rfm_code)
        segment_label = f"{consumo_label} + {rfm_label}"

        print("\n" + "-"*60)
        print(f"### Segmento Combinado: {segment_label} ###")
        print("-"*60)

        # Filtra o segmento desejado
        df_segment = dfp_full.filter(
            (col("consumption_segment") == consumo_code) &
            (col("rfm_segment") == rfm_code)
        )

        if df_segment.count() < 100:  # Pode ajustar esse número
            print("⚠️  Amostra insuficiente para análise robusta. Pulando...")
            continue

        # Conversões para teste estatístico
        df_segment = df_segment.withColumn("is_retained_int", col("is_retained").cast("integer"))
        df_segment = StringIndexer(inputCol="is_target", outputCol="label").fit(df_segment).transform(df_segment)
        df_segment = VectorAssembler(inputCols=["is_retained_int"], outputCol="features").transform(df_segment)

        # Teste Qui-Quadrado
        chi_square_result = ChiSquareTest.test(df_segment, "features", "label").first()
        p_value = float(chi_square_result.asDict()["pValues"][0])
        chi2_statistic = float(chi_square_result.asDict()["statistics"][0])

        print(f"P-Valor: {p_value:.8f}")
        print(f"Estatística Qui-Quadrado: {chi2_statistic:.4f}")
        if p_value < 0.05:
            print("✅ Diferença estatisticamente significativa.")
        else:
            print("❌ Diferença NÃO estatisticamente significativa.")

        # Métricas de negócio
        summary_df = df_segment.groupBy("is_target").agg(
            count("*").alias("total_users"),
            count(when(col("is_retained") == True, True)).alias("retained_users")
        ).collect()

        control = [r for r in summary_df if r["is_target"] == "control"][0]
        target = [r for r in summary_df if r["is_target"] == "target"][0]

        control_total = control["total_users"]
        control_retained = control["retained_users"]
        target_total = target["total_users"]
        target_retained = target["retained_users"]

        rate_control = control_retained / control_total
        rate_target = target_retained / target_total
        lift = ((rate_target - rate_control) / rate_control) * 100 if rate_control > 0 else 0
        diff = rate_target - rate_control

        std_dev1_sq = rate_target * (1 - rate_target) / target_total
        std_dev2_sq = rate_control * (1 - rate_control) / control_total
        std_err_diff = math.sqrt(std_dev1_sq + std_dev2_sq)
        conf_interval_diff = (diff - 1.96 * std_err_diff, diff + 1.96 * std_err_diff)

        print(f"\n--- Métricas de Negócio ---")
        print(f"Taxa de Retenção (Controle): {rate_control:.2%}")
        print(f"Taxa de Retenção (Teste): {rate_target:.2%}")
        print(f"Lift Percentual: {lift:.2f}%")
        print(f"Diferença Absoluta: {diff:.2%}")
        print(f"Intervalo de Confiança 95%: [{conf_interval_diff[0]:.2%}, {conf_interval_diff[1]:.2%}]")

        # Armazenamento
        final_cross_results.append({
            "Segmento Combinado": segment_label,
            "Retenção (Controle)": f"{rate_control:.2%}",
            "Retenção (Teste)": f"{rate_target:.2%}",
            "Lift (%)": f"{lift:.2f}%",
            "p-value": p_value,
            "IC 95%": f"[{conf_interval_diff[0]:.2%}, {conf_interval_diff[1]:.2%}]"
        })

# Mostrar resultados finais como tabela
df_result = pd.DataFrame(final_cross_results)
print("\n\n" + "="*80)
print("### TABELA FINAL: Análise de Impacto por Segmento Combinado ###")
print("="*80)
print(df_result.to_string(index=False))




### ANÁLISE FINAL: SEGMENTOS COMBINADOS (PADRÃO DE CONSUMO + RFM) ###

------------------------------------------------------------
### Segmento Combinado: Usuário de Fim de Semana + Em Risco / Hibernando ###
------------------------------------------------------------
P-Valor: 0.00000000
Estatística Qui-Quadrado: 615.0874
✅ Diferença estatisticamente significativa.

--- Métricas de Negócio ---
Taxa de Retenção (Controle): 38.65%
Taxa de Retenção (Teste): 49.01%
Lift Percentual: 26.80%
Diferença Absoluta: 10.36%
Intervalo de Confiança 95%: [9.54%, 11.17%]

------------------------------------------------------------
### Segmento Combinado: Usuário de Fim de Semana + Outros ###
------------------------------------------------------------
P-Valor: 0.00000000
Estatística Qui-Quadrado: 137.0084
✅ Diferença estatisticamente significativa.

--- Métricas de Negócio ---
Taxa de Retenção (Controle): 4.49%
Taxa de Retenção (Teste): 6.65%
Lift Percentual: 47.97%
Diferença Absoluta: 2.16%
Interv

In [25]:
df_filtered_full = dfp_full.filter(((col("consumption_segment") == '1') | (col("consumption_segment") == '2')) & ((col("rfm_segment") == '3') | (col("rfm_segment") == '5')))


dfp_total_customers_per_group = df_filtered_full.groupBy("is_target","is_retained","consumption_segment","rfm_segment").agg(
    count("customer_id").alias("total_customers")
)
dfp_total_customers_per_group.show()

+---------+-----------+-------------------+-----------+---------------+
|is_target|is_retained|consumption_segment|rfm_segment|total_customers|
+---------+-----------+-------------------+-----------+---------------+
|   target|       true|                  2|          3|          58920|
|  control|      false|                  2|          3|          39238|
|   target|      false|                  1|          5|          29351|
|  control|      false|                  1|          5|          29347|
|   target|       true|                  1|          3|          15098|
|  control|       true|                  1|          3|          10108|
|  control|      false|                  1|          3|          16045|
|   target|      false|                  2|          3|          39512|
|  control|      false|                  2|          5|          32656|
|   target|      false|                  2|          5|          32793|
|  control|       true|                  2|          3|         

## Conclusão: Impacto da Campanha em Segmentos Combinados (Padrão de Consumo + RFM)

Com base na análise estatística e nos indicadores de negócio gerados a partir da combinação entre os segmentos de **padrão de consumo** e **perfil RFM**, foi possível identificar interseções que se destacam tanto pelo impacto da campanha quanto pelo potencial de ação estratégica.

### Segmento: Usuário de Fim de Semana + Em Risco / Hibernando
- **Taxa de Retenção (Controle):** 38.65%
- **Taxa de Retenção (Teste):** 49.01%
- **Lift Percentual:** +26.80%
- **Diferença Absoluta:** +10.36 pontos percentuais
- **Intervalo de Confiança 95%:** [9.54%, 11.17%]

Este grupo une dois perfis já validados individualmente como responsivos: usuários em risco e com forte comportamento de uso concentrado nos fins de semana. A campanha demonstrou um impacto substancial e estatisticamente significativo, indicando alto potencial de retorno com estratégias específicas de reengajamento nesse perfil. Abordagens com gatilhos personalizados em dias estratégicos podem amplificar ainda mais os resultados.

### Segmento: Usuário de Dias de Semana + Em Risco / Hibernando
- **Taxa de Retenção (Controle):** 50.20%
- **Taxa de Retenção (Teste):** 59.86%
- **Lift Percentual:** +19.25%
- **Diferença Absoluta:** +9.66 pontos percentuais
- **Intervalo de Confiança 95%:** [9.20%, 10.13%]

Assim como seu correspondente de fim de semana, este segmento confirmou o bom desempenho da campanha entre perfis com risco de churn, especialmente os que interagem nos dias úteis. Campanhas de reativação integradas à rotina semanal — como notificações durante a semana ou benefícios em dias úteis — podem ser chave para manter esse público engajado.

### Segmento: Usuário de Fim de Semana + Outros
- **Taxa de Retenção (Controle):** 4.49%
- **Taxa de Retenção (Teste):** 6.65%
- **Lift Percentual:** +47.97%
- **Diferença Absoluta:** +2.16 pontos percentuais
- **Intervalo de Confiança 95%:** [1.80%, 2.52%]

Apesar da base relativamente pequena, o lift percentual foi expressivo. O grupo “Outros” pode refletir perfis menos consistentes ou em fase de descoberta do app. A campanha pode ter atuado como ponto de entrada ou reforço inicial para consolidar o uso. Estratégias como onboarding otimizado, incentivos progressivos ou tutoriais gamificados podem ser especialmente eficazes nesse público.

### Segmento: Usuário de Dias de Semana + Outros
- **Taxa de Retenção (Controle):** 4.72%
- **Taxa de Retenção (Teste):** 6.85%
- **Lift Percentual:** +45.16%
- **Diferença Absoluta:** +2.13 pontos percentuais
- **Intervalo de Confiança 95%:** [1.78%, 2.48%]

Perfil semelhante ao anterior, mas com comportamento de uso durante os dias úteis. A boa resposta à campanha reforça a hipótese de que este grupo reúne usuários em fase inicial ou com uso esporádico. Estratégias leves e recorrentes de ativação podem funcionar como mecanismos de transição para perfis mais fiéis.

---

## Conclusão Geral

A análise combinada mostra que a intersecção entre padrões de uso e perfis de relacionamento pode revelar **segmentos de alto valor estratégico**. Dois grandes grupos se destacam:

- **Em Risco / Hibernando + qualquer padrão de consumo:** Resposta sólida à campanha, com impacto robusto e replicável. Estratégias de reengajamento personalizadas são recomendadas.
- **Segmento “Outros”:** Apesar das baixas taxas iniciais, esse grupo mostrou o maior *lift percentual*, sinalizando que a campanha pode estar atuando como uma porta de entrada para consolidar o uso do app. Investir nesse perfil pode representar **uma oportunidade de conversão de clientes indecisos em usuários ativos**.

Com esses achados, torna-se evidente que **ações personalizadas por perfis compostos podem aumentar substancialmente a eficácia de campanhas de retenção**, orientando investimentos e comunicações de forma mais precisa e orientada por dados.


In [ ]:
def sink_data_gold(df,gold_path):
  df.write.mode("overwrite").parquet(gold_path)


In [ ]:
drive_base_path = "drive/MyDrive/iFood/Data/"

if data_source == "drive":
    # Define o caminho para a camada 'Gold' dentro da estrutura do Drive.
    dfp_orders_filtered_gold = f"{drive_base_path}/Gold/dfp_orders_filtered.parquet"
    dfp_analysis_gold = f"{drive_base_path}/Gold/dfp_analysis.parquet"
    dfp_master_consumption_gold = f"{drive_base_path}/Gold/dfp_master_consumption.parquet"
    dfp_master_rfm_gold = f"{drive_base_path}/Gold/dfp_master_rfm.parquet"
    dfp_final_segmentation_rfm_gold = f"{drive_base_path}/Gold/dfp_final_segmentation_rfm.parquet"
    dfp_final_segmentation_pattern_gold = f"{drive_base_path}/Gold/dfp_final_segmentation_pattern.parquet"
    df_result_gold_path = f"{drive_base_path}/Gold/df_result_consumption.parquet"
    df_final_biz_gold_path = f"{drive_base_path}/Gold/df_result_rfm.parquet"
    df_final_biz_rfm_gold_path = f"{drive_base_path}/Gold/df_result_rfm_final.parquet"



sink_data_gold(dfp_orders_filtered, dfp_orders_filtered_gold)
sink_data_gold(dfp_analysis, dfp_analysis_gold)
sink_data_gold(dfp_master_consumption, dfp_master_consumption_gold)
sink_data_gold(dfp_master_rfm, dfp_master_rfm_gold)
sink_data_gold(dfp_final_segmentation_rfm, dfp_final_segmentation_rfm_gold)
sink_data_gold(dfp_final_segmentation_pattern, dfp_final_segmentation_pattern_gold)
df_result.to_parquet(df_result_gold_path)
df_final_biz.to_parquet(df_final_biz_gold_path)
df_final_biz_rfm.to_parquet(df_final_biz_rfm_gold_path)

